# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

* Import the `establishments.json` file with the following line in Terminal:

    ```shell
    mongoimport --type csv -d uk_food -c establishments --headerline --drop establishments.json
    ```
* Run `mongo` from the Terminal to launch Mongo.

* Display the list of collections in the database:

    ```shell
    show collections
    ```

In [67]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [68]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [69]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [70]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [71]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [72]:
# review a document in the establishments collection
print(db.establishments.find_one())


{'_id': ObjectId('64832f7ed6e509bfb2f3656f'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': {'$toDouble': '$geocode.longitude'}, 'latitude': {'$toDouble': '$geocode.latitude'}}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extra

In [73]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [74]:
# Create a dictionary for the new restaurant data
halal = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [75]:
# Insert the new restaurant into the collection
establishments.insert_one(halal)

In [76]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishments.find(query)
for result in results:
    pprint(results)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [77]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': "Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID'}
results = establishments.find_one(query, fields)
pprint(results)

{'BusinessTypeID': 1, '_id': ObjectId('64832f7ed6e509bfb2f3656f')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [78]:
# Update the new restaurant with the correct BusinessTypeID
filter = { 'BusinessName': 'Penang Flavours' }
        
        # Specify the update operation
update = {
    '$set': {
        'BusinessTypeID': '1',
    }
}
establishments.update_one(filter, update)

In [79]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
pprint(establishments.find_one(query))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6487b9bab313009564cb9853'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'latitude': {'$toDouble': '$geocode.latitude'},
 'longitude': {'$toDouble': '$geocode.longitude'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [80]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.count_documents(query)

0

In [81]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [82]:
# Check if any remaining documents include Dover
establishments.count_documents(query)

0

In [83]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('64832f7ed6e509bfb2f3656f'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': {'$toDouble': '$geocode.longitude'},
  'latitude': {'$toDouble': '$geocode.latitude'}},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta':

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [90]:
# Change the data type from String to Decimal for longitude and latitude
filter = {}
        
        # Specify the update operation
update = [{
    '$set': {
        'geocode.longitude': {'$toDouble': '$geocode.longitude'},
        'geocode.latitude': {'$toDouble': '$geocode.latitude'}
    }
}]
establishments.update_many(filter, update)
establishments.find_one()

WriteError: Unsupported conversion from object to double in $convert with no onError value, full error: {'index': 0, 'code': 241, 'errmsg': 'Unsupported conversion from object to double in $convert with no onError value'}

Use `update_many` to convert `RatingValue` to integer numbers.

In [85]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [88]:
# Change the data type from String to Integer for RatingValue
filter = {}
        
        # Specify the update operation
update = [{
    '$set': {
        'RatingValue': {'$toInt': '$RatingValue'},
    }
}]
establishments.update_many(filter, update)

In [89]:
# Check that the coordinates and rating value are now numbers
establishments.find_one()

{'_id': ObjectId('64832f7ed6e509bfb2f3656f'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': {'$toDouble': '$geocode.longitude'},
  'latitude': {'$toDouble': '$geocode.latitude'}},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPe